In [9]:
import os
import re
import zipfile
from langchain_groq import ChatGroq
from langchain import PromptTemplate
from langchain_core.rate_limiters import InMemoryRateLimiter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter, LLMChainExtractor, LLMChainFilter
from langchain.chains import create_retrieval_chain
from langchain_community.document_loaders.csv_loader import CSVLoader
from tqdm import tqdm
import pandas as pd
import logging
logger = logging.getLogger()

In [3]:
data_path = "./pc-part-dataset/data/"
# dataset_path = os.path.join(folder_path, "dataset.zip")
# test_data_path = os.path.join(folder_path, "LR2_dev.csv")
# test_answers_path = os.path.join(folder_path, "LR2_dev_answer.csv")
# eval_data_path = os.path.join(folder_path, "LR2.csv")

In [4]:
# emb_model1 = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", model_kwargs = {'device': 'mps'})
emb_model1 = HuggingFaceEmbeddings(
    model_name="cointegrated/LaBSE-en-ru", # Более качественные русско-английские эмбеддинги
    model_kwargs={'device': 'mps'},
    encode_kwargs={'normalize_embeddings': True} # Важно для LaBSE
)

/var/folders/1w/zvs_tqsn6gd7ls8fcvqf0n340000gn/T/ipykernel_80365/801422192.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb_model1 = HuggingFaceEmbeddings(
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Генерация векторной базы

In [5]:
# os.environ["GROQ_API_KEY"] = "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" # main
os.environ["GROQ_API_KEY"] = "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" # second

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.5,  # Controls requests per second
    check_every_n_seconds=0.8,  # How often to check rate limit
    max_bucket_size=5  # Maximum burst size allowed
)


llm = ChatGroq(model="llama3-70b-8192", rate_limiter=rate_limiter, temperature=0.6)

In [6]:
# from langchain_community.agent_toolkits import SQLDatabaseToolkit
# from langchain_community.utilities import SQLDatabase

# db = SQLDatabase.from_uri(PG_DSN)
# toolkit = SQLDatabaseToolkit(db=db, llm=llm)

# tools = toolkit.get_tools()

# tools

In [10]:
# from langchain import hub

# prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

# assert len(prompt_template.messages) == 1
# prompt_template.messages[0].pretty_print()

In [ ]:
# system_message = prompt_template.format(dialect="Postres", top_k=5)

In [ ]:
# from langchain_core.messages import HumanMessage
# from langgraph.prebuilt import create_react_agent

# agent_executor = create_react_agent(llm, tools, prompt=system_message)

In [ ]:
# llm.invoke("Key characteristics of best and powerful gpu").content

"Here are the key characteristics of a powerful and high-performance GPU:\n\n1. **High Clock Speed**: A high clock speed, measured in GHz, indicates how fast the GPU can perform calculations. A higher clock speed generally results in better performance.\n2. **Number of CUDA Cores/Stream Processors**: More CUDA cores/stream processors mean the GPU can handle more complex calculations and process more data simultaneously, leading to better performance.\n3. **Memory Bandwidth**: Higher memory bandwidth, measured in GB/s, allows the GPU to access and process more data quickly, resulting in better performance.\n4. **Memory Size and Type**: A larger and faster memory (e.g., GDDR6 or HBM2) can store more data and provide faster access, leading to better performance.\n5. **Memory Interface Width**: A wider memory interface width (e.g., 384-bit or 512-bit) allows for more data to be transferred simultaneously, resulting in better performance.\n6. **Pixel Fill Rate**: A higher pixel fill rate, m

In [11]:
# question = "(Table video-card) Best and powerful 5 gpus under 500$ (Here are the key characteristics of a powerful and high-performance GPU:\n\n1. **High Clock Speed**: A high clock speed, measured in GHz, indicates how fast the GPU can perform calculations. A higher clock speed generally results in better performance.\n2. **Number of CUDA Cores/Stream Processors**: More CUDA cores/stream processors mean the GPU can handle more complex calculations and process more data simultaneously, leading to better performance.\n3. **Memory Bandwidth**: Higher memory bandwidth, measured in GB/s, allows the GPU to access and process more data quickly, resulting in better performance.\n4. **Memory Size and Type**: A larger and faster memory (e.g., GDDR6 or HBM2) can store more data and provide faster access, leading to better performance.\n5. **Memory Interface Width**: A wider memory interface width (e.g., 384-bit or 512-bit) allows for more data to be transferred simultaneously, resulting in better performance.\n6. **Pixel Fill Rate**: A higher pixel fill rate, measured in gigapixels per second, indicates how quickly the GPU can render pixels, leading to smoother graphics and better performance.\n7. **Texture Mapping Units (TMUs)**: More TMUs allow the GPU to handle more complex textures and shading, resulting in better graphics quality and performance.\n8. **Render Outputs (ROPs)**: More ROPs enable the GPU to render more pixels and handle more complex graphics, leading to better performance.\n9. **PCIe Version**: A newer PCIe version (e.g., PCIe 4.0) provides faster data transfer rates between the GPU and system, resulting in better performance.\n10. **Power Consumption**: A higher power consumption, measured in watts, can indicate a more powerful GPU, but also increases heat generation and power draw.\n11. **Cooling System**: An efficient cooling system, such as a hybrid or liquid cooling system, helps to keep the GPU at a safe temperature, ensuring stable performance and longevity.\n12. **DirectX and Vulkan Support**: Support for the latest DirectX and Vulkan APIs ensures compatibility with modern games and applications, taking advantage of the GPU's features and performance.\n13. **Multi-GPU Support**: Support for multi-GPU configurations, such as SLI or Crossfire, allows for increased performance and scalability in compatible systems.\n14. **Artificial Intelligence (AI) and Machine Learning (ML) Acceleration**: Integration of AI and ML acceleration, such as Tensor Cores or Stream processors, enables faster processing of AI and ML workloads.\n15. **Ray Tracing and AI-Enhanced Graphics**: Support for real-time ray tracing and AI-enhanced graphics, such as NVIDIA's RTX or AMD's Radeon RDNA 2, enables more realistic and detailed graphics rendering.\n16. **Display Outputs and Resolution**: Multiple display outputs and support for high resolutions (e.g., 4K or 8K) enable more flexible and high-quality display configurations.\n17. **HDMI and DisplayPort Versions**: Support for the latest HDMI and DisplayPort versions ensures compatibility with modern displays and enables features like variable refresh rate and HDR.\n18. **Graphics Processing Unit (GPU) Architecture**: A modern and efficient GPU architecture, such as NVIDIA's Ampere or AMD's RDNA 2, enables better performance, power efficiency, and feature support.\n\nWhen evaluating a GPU, consider your specific needs and priorities. For gaming, focus on clock speed, memory bandwidth, and pixel fill rate. For professional applications like video editing or 3D modeling, consider memory size, memory bandwidth, and CUDA core/stream processor count)"

# question = "Best and powerful gpus under 40000 RUB"

# for step in agent_executor.stream(
#     {"messages": [{"role": "user", "content": question}]},
#     stream_mode="values",
# ):
#     step["messages"][-1].pretty_print()

# Остальное

In [12]:
folder_path = './pc-part-dataset/data/csv'
tables = ["case-fan.csv", "case.csv", "cpu-cooler.csv", "cpu.csv", "internal-hard-drive.csv", "memory.csv", "motherboard.csv", "power-supply.csv", "video-card.csv"]
tables_names = ["case_fan", "case", "cpu-cooler", "cpu", "drive", "memory", "motherboard", "power-supply", "gpu"]

In [13]:
data = []
for filename in tables:
    loader = CSVLoader(os.path.join(folder_path, filename))
    data.extend(loader.load())

In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=128,
    separators=["\n\n", "\n", r"(?<=\. )", " ", ""],
    length_function=len,
    is_separator_regex=True,
)

In [11]:
retrv = {}
for k in file_to_book.values():
    retrv[k] = ""

NameError: name 'file_to_book' is not defined

In [ ]:
for doc in docs:
    if doc.metadata["source"].split("/")[-1] not in file_to_book.keys():
        continue
    data = doc.page_content
    split_data = splitter.create_documents([data])
    db = FAISS.from_documents(split_data, emb_model1)
    retrv[file_to_book[doc.metadata["source"].split("/")[-1]]] = db

In [ ]:
retrievers = {}
for k, v in retrv.items():
    retrievers[k] = v.as_retriever(
    search_type="mmr",  # Maximum Marginal Relevance search
    # search_type="similarity",
    search_kwargs={
        'k': 5,  # Number of documents to retrieve
        'fetch_k': 15,  # Total documents to consider
        'lambda_mult': 0.5  # Balance between diversity and relevance
    }
)